In [1]:
import pandas as pd
import os
from src.utils.results_utils import mapper
from fuzzywuzzy import fuzz, process

In [2]:
folder = "static"
df = pd.DataFrame()
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        try:
            temp = pd.read_excel(os.path.join(folder, file), sheet_name="Data")
        except:
            temp = pd.read_excel(os.path.join(folder, file), sheet_name="Sheet 1")
        df = pd.concat([df, temp])


In [4]:
accepted_df = pd.read_excel("../../MV Maids All Steps.xlsx", sheet_name="Sheet 1")

In [5]:
accepted_df.ffill(inplace=True)
df.ffill(inplace=True)

In [6]:
ids = accepted_df[["Maid ID", "Maid Name"]]

In [7]:
target = df[["Maid’s ID", "Modified Field", "Maid’s Nationality", "Agent Value"]]

In [8]:
target = target.merge(ids, how="inner", left_on="Maid’s ID", right_on="Maid ID")

In [9]:
places = target[target["Modified Field"] == "Birth Place"]

In [28]:
counts = places.drop_duplicates(subset=["Maid’s Nationality", "Agent Value", "Maid’s ID"])["Agent Value"].value_counts()

In [23]:
places[places["Agent Value"] == "WOLAYETA"] 

,Maid’s ID,Modified Field,Maid’s Nationality,Agent Value,Maid ID,Maid Name
1034657,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
1034658,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
1034659,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
1034660,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
1034661,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
...,...,...,...,...,...,...
2508252,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
2508253,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
2508254,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE
2508255,77581.0,Birth Place,Ethiopian,WOLAYETA,77581.0,BELAYNESH BEKELE YOTE


In [13]:
# Get unique places and nationalities
places_unique = places[["Maid’s ID", "Maid’s Nationality", "Agent Value"]].drop_duplicates(subset=["Maid’s Nationality", "Agent Value"])

# Normalize nationality to country code using mapper
def get_country_code(nationality):
    # Use fuzzy matching to find the closest key in mapper
    matched_country, score = process.extractOne(nationality, mapper.keys())
    return mapper.get(matched_country, "XXX")

places_unique["country_code"] = places_unique["Maid’s Nationality"].apply(get_country_code)

# Prepare DataFrame for custom_cities.csv format
places_to_add = places_unique[["Agent Value", "country_code"]].rename(columns={"Agent Value": "name"})

len(places_to_add)

2099

In [29]:
places_with_counts = places_to_add.merge(
    right=counts,
    left_on="name",
    right_on="Agent Value"
)

In [30]:
# sort by country code then count
places_with_counts.sort_values(by=["country_code", "count"], ascending=[True, False], inplace=True)

In [31]:
places_with_counts.to_csv("issueplaces_with_counts.csv", index=False)

In [16]:
places_to_add.to_csv("birthplaces.csv", index=False)

In [22]:
places = target[target["Modified Field"] == "Passport Place(EN)"]

In [23]:
places.drop_duplicates(subset=["Maid’s Nationality", "Agent Value", "Maid’s ID"])["Agent Value"].value_counts().head(50)

Agent Value
PCG DUBAI                619
MOFA                     215
DFA MANILA               214
PE ABU DHABI             204
GOVERNMENT OF KENYA      170
ETHIOPIA                 169
COLOMBO                  140
UGANDA GOVT KAMPALA      116
COCHIN                    86
REGISTRAR GENERAL HRE     59
DUBAI                     54
PE RIYADH                 42
DFA DAVAO                 41
TRIVANDRUM                41
DFA PAMPANGA              38
DFA ILOILO                37
PCG HONG KONG             35
JALANDHAR                 35
DFA GENERAL SANTOS        34
BENGALURU                 34
DFA NCR SOUTH             33
KOLKATA                   33
DFA NCR WEST              30
DFA BACOLOD               29
DFA LUCENA                27
DFA TUGUEGARAO            26
DFA NCR NORTHEAST         24
DFA NCR NORTH             23
DFA SANTIAGO CITY         23
DFA LA UNION              23
PE DOHA                   22
KOZHIKODE                 22
PE KUWAIT                 21
DFA ANGELES               21
DF

In [24]:
# Get unique places and nationalities
places_unique = places[["Maid’s ID", "Maid’s Nationality", "Agent Value"]].drop_duplicates(subset=["Maid’s Nationality", "Agent Value"])

# Normalize nationality to country code using mapper
def get_country_code(nationality):
    # Use fuzzy matching to find the closest key in mapper
    matched_country, score = process.extractOne(nationality, mapper.keys())
    return mapper.get(matched_country, "XXX")

places_unique["country_code"] = places_unique["Maid’s Nationality"].apply(get_country_code)

# Prepare DataFrame for custom_cities.csv format
places_to_add = places_unique[["Agent Value", "country_code"]].rename(columns={"Agent Value": "name"})

len(places_to_add)

276

In [20]:
places_to_add.to_csv("issueplaces.csv", index=False)

In [42]:
# Identify similar city names within each country code
from collections import defaultdict

# Set similarity threshold
similarity_threshold = 95  # Adjust as needed (higher = stricter matching)

# Group places by country code
country_groups = places_to_add.groupby('country_code')

# Dictionary to store similar place clusters for each country
similar_places = defaultdict(list)

# Process each country's places
for country_code, group in country_groups:
    places_list = group['name'].tolist()
    
    # Skip processing if too few places
    if len(places_list) < 2:
        continue
    
    # Find similar places within this country
    processed = set()
    
    for i, place1 in enumerate(places_list):
        if place1 in processed:
            continue
            
        cluster = [place1]
        processed.add(place1)
        
        for place2 in places_list[i+1:]:
            if place2 in processed:
                continue
                
            # Calculate similarity ratio
            similarity = fuzz.ratio(place1, place2)
            
            # If similar enough, add to cluster
            if similarity >= similarity_threshold:
                cluster.append(place2)
                processed.add(place2)
        
        # Store cluster if it contains multiple places
        if len(cluster) > 1:
            similar_places[country_code].append(cluster)

place_counts = places_to_add['name'].value_counts().to_dict()

# Create a DataFrame with all similar place clusters for easier inspection
similar_df_rows = []
country = "PHL"
clusters = similar_places[country]

for cluster in clusters:
    for place in cluster:
        print(place, place_counts.get(place, 0), place_ids[place_ids['name'] == place]['Maid’s ID'].tolist(), end=" ")
    print()

G GENEROSO DV OR 1 [[85982.0]] G GENEROSO DVO OR 1 [[75564.0]] 
SN FERNANDO CEBU 1 [[77522.0]] SAN FERNANDO CEBU 1 [[89837.0]] 
MATALAM NO COT 1 [[86862.0]] MATALAM N COT 1 [[88297.0]] 
SANCHEZ MIRA CAG 1 [[86967.0]] SANCHEZ MIRA CAGE 1 [[92705.0]] 
ALLANERA N ECIJA 1 [[86928.0]] LLANERA N ECIJA 1 [[84366.0]] 
KIDAPAWAN NO COT 1 [[84771.0]] KIDAPAWAN NO CO 1 [[91712.0]] 
SAN FERNANDO PMP 1 [[85775.0]] SAN FERNANDO PAMP 1 [[87102.0]] 
SAN LUIS PAMPANGA 1 [[87416.0]] SN LUIS PAMPANGA 1 [[85282.0]] 
LAGUINDINGAN MO 1 [[85280.0]] LAGUINDINGAN M O 1 [[86057.0]] 
BANGA S COTABATO 1 [[84942.0]] BANGA SO COTABATO 1 [[91688.0]] 
GAPAN N ECIJA 1 [[80319.0]] GAPAN NECIJA 1 [[86087.0]] 
KALIBO AKLAN 1 [[80321.0]] KALIBO KLAN 1 [[80321.0]] 
VALENCIA BKN 1 [[82897.0]] VALECIA BKN 1 [[93211.0]] 
MINGLANILA CEBU 1 [[80334.0]] MINGLANILLA CEBU 1 [[84856.0]] 
LAPU LAPU CEBU 2 [[84450.0]] LAPULAPU CEBU 1 [[81000.0]] 
APARRI CAGAYAN 1 [[87029.0]] APARRI AGAYAN 1 [[82652.0]] 
DUPAX DN N VIZ 2 [[83243.0]] D

In [ ]:
# After reviewing similar_places.csv, create a mapping of misspelled names to correct names
# For example:
corrections = {
    # 'MISSPELLED': 'CORRECT',
    # 'NAIROBE': 'NAIROBI',
    # Add your corrections here
    "ADDIS ABEBA": "ADDIS ABABA",
    "NABUNTURAN DDN": "NABUNTURAN DON",
    "DAGUPAN CI": "DAGUPAN CITY",
    "STO TOMAS DVO": "STO TOMAS DAVAO",
    "STO TOMAS DVO DN": "STO TOMAS DAVAO",
    "STO TOMAS DAVAD": "STO TOMAS DAVAO",
    "STO TOMAS DVD": "STO TOMAS DAVAO",
    "S-BULLONES BOHO": "S BULLONES BOHO",
}

# Apply corrections to the places_to_add DataFrame
places_to_add['name'] = places_to_add['name'].apply(lambda x: corrections.get(x, x))

# Remove duplicates after corrections
places_to_add = places_to_add.drop_duplicates(subset=['name', 'country_code'])

# Update combined_cities with corrected names
combined_cities = pd.concat([existing_cities, places_to_add]).drop_duplicates(subset=['name'])

# Save updated custom_cities.csv
combined_cities.to_csv(custom_cities_path, index=False)

print(f"Updated custom_cities.csv with corrections")